<a href="https://colab.research.google.com/github/Lokendra-parmar/python-programming-questions/blob/main/MLPproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load all necessary files
try:
    booknow_booking_df = pd.read_csv('booknow_booking.csv')
    cinepos_booking_df = pd.read_csv('cinePOS_booking.csv')
    id_relation_df = pd.read_csv('movie_theater_id_relation.csv')
    booknow_visits_df = pd.read_csv('booknow_visits.csv')
    date_info_df = pd.read_csv('date_info.csv')
    booknow_theaters_df = pd.read_csv('booknow_theaters.csv')

    print("All files loaded successfully.")
except Exception as e:
    print(f"Error loading files: {e}")
    # Stop execution if files can't be loaded
    raise e

print("Starting data consolidation...")

# --- 1a: Clean booknow_theaters ---
# Drop rows where book_theater_id is null, as they cannot be linked
booknow_theaters_df.dropna(subset=['book_theater_id'], inplace=True)
# We'll ignore lat/lon and sparse theater_type/area for this model
booknow_theaters_df = booknow_theaters_df[['book_theater_id']]

# --- 1b: Process booknow_booking (Online) ---
# Convert to datetime and get the date part
booknow_booking_df['show_datetime'] = pd.to_datetime(booknow_booking_df['show_datetime'])
booknow_booking_df['show_date'] = booknow_booking_df['show_datetime'].dt.strftime('%Y-%m-%d')
# Aggregate: sum tickets by theater and date
booknow_agg_df = booknow_booking_df.groupby(['book_theater_id', 'show_date'])['tickets_booked'].sum().reset_index()
booknow_agg_df.rename(columns={'tickets_booked': 'total_booknow_tickets'}, inplace=True)

# --- 1c: Process cinePOS_booking (On-site) ---
# Convert to datetime and get the date part
cinepos_booking_df['show_datetime'] = pd.to_datetime(cinepos_booking_df['show_datetime'])
cinepos_booking_df['show_date'] = cinepos_booking_df['show_datetime'].dt.strftime('%Y-%m-%d')
# Aggregate: sum tickets by theater and date
cinepos_agg_df = cinepos_booking_df.groupby(['cine_theater_id', 'show_date'])['tickets_sold'].sum().reset_index()
cinepos_agg_df.rename(columns={'tickets_sold': 'total_cinepos_tickets'}, inplace=True)

# --- 1d: Link cinePOS to booknow IDs ---
cinepos_linked_df = pd.merge(cinepos_agg_df, id_relation_df, on='cine_theater_id', how='inner')
# Re-aggregate in case multiple cinePOS IDs map to a single book_theater_id
cinepos_linked_agg_df = cinepos_linked_df.groupby(['book_theater_id', 'show_date'])['total_cinepos_tickets'].sum().reset_index()

# --- 1e: Create Master DataFrame ---
# Start with the base visits data (our target)
master_df = booknow_visits_df.copy()

# Merge calendar info
master_df = pd.merge(master_df, date_info_df, on='show_date', how='left')

# Merge aggregated BookNow bookings
master_df = pd.merge(master_df, booknow_agg_df, on=['book_theater_id', 'show_date'], how='left')

# Merge aggregated and linked CinePOS bookings
master_df = pd.merge(master_df, cinepos_linked_agg_df, on=['book_theater_id', 'show_date'], how='left')

# --- 1f: Final Cleanup ---
# Fill booking NaNs with 0 (days with visits but no recorded online/POS bookings)
master_df['total_booknow_tickets'].fillna(0, inplace=True)
master_df['total_cinepos_tickets'].fillna(0, inplace=True)

# Convert show_date to datetime object for sorting and feature engineering
master_df['show_date'] = pd.to_datetime(master_df['show_date'])

print("--- Master DataFrame Created ---")
print(master_df.head())
print(f"\nShape of master_df: {master_df.shape}")
print(master_df.info())

Step 2: Feature Engineering & Model Validation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

print("\n--- Starting Step 2: Feature Engineering & Validation ---")

# --- 2a: Create Features ---
# Create 'total_tickets' feature
master_df['total_tickets'] = master_df['total_booknow_tickets'] + master_df['total_cinepos_tickets']

# Date Features
master_df['day_of_month'] = master_df['show_date'].dt.day
master_df['month'] = master_df['show_date'].dt.month
master_df['year'] = master_df['show_date'].dt.year
master_df['day_of_year'] = master_df['show_date'].dt.dayofyear
master_df['is_weekend'] = master_df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)

# CRITICAL: Sort by theater and date
master_df = master_df.sort_values(by=['book_theater_id', 'show_date'])

# Lag & Rolling Features
print("Creating lag and rolling features...")
gb = master_df.groupby('book_theater_id')['audience_count']
master_df['audience_lag_7'] = gb.shift(7)
master_df['audience_lag_14'] = gb.shift(14)
master_df['audience_roll_mean_7'] = gb.shift(1).rolling(7, min_periods=1).mean()

# --- 2b: Categorical Encoding ---
# We will use LabelEncoder for IDs and One-Hot Encoding for 'day_of_week'
le = LabelEncoder()
master_df['book_theater_id_encoded'] = le.fit_transform(master_df['book_theater_id'])
master_df = pd.get_dummies(master_df, columns=['day_of_week'], prefix='dow')

# --- 2c: Clean Data ---
# Drop rows where lag features are NaN (at the start of each series)
master_df_cleaned = master_df.dropna()
print(f"Data shape after feature engineering and cleaning: {master_df_cleaned.shape}")

# --- 2d: Time-Series Split for Validation ---
target_col = 'audience_count'
# Exclude original IDs and date
features = [col for col in master_df_cleaned.columns if col not in [
    'audience_count', 'show_date', 'book_theater_id'
]]

X = master_df_cleaned[features]
y = master_df_cleaned[target_col]

# We will use the last 4 weeks (28 days) for validation
max_date = master_df_cleaned['show_date'].max()
split_date = max_date - pd.to_timedelta('28 days')

train_mask = (master_df_cleaned['show_date'] < split_date)
valid_mask = (master_df_cleaned['show_date'] >= split_date)

X_train, y_train = X[train_mask], y[train_mask]
X_valid, y_valid = X[valid_mask], y[valid_mask]

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_valid.shape}")

# --- 2e: Train and Validate Model ---
print("\nTraining RandomForestRegressor for validation...")
# Use a fast and powerful RandomForest
rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=5,
    max_features=0.7
)

rf.fit(X_train, y_train)

# Evaluate
y_pred = rf.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"\n--- Validation Complete ---")
print(f"Validation RMSE: {rmse:.4f}")
print("This shows our model is predictive. Now we will build the final submission.")

Step 3: Create Full Dataset for Submission

In [ ]:
import pandas as pd
from itertools import product
from sklearn.preprocessing import LabelEncoder

print("\n--- Starting Step 3: Creating Full Train+Test Dataset ---")

# --- 3a: Reload original data ---
# We need the original files to build the full train+test set
booknow_visits_df = pd.read_csv('booknow_visits.csv')
date_info_df = pd.read_csv('date_info.csv')

# --- 3b: Identify Test Period ---
booknow_visits_df['show_date'] = pd.to_datetime(booknow_visits_df['show_date'])
date_info_df['show_date'] = pd.to_datetime(date_info_df['show_date'])
max_train_date = booknow_visits_df['show_date'].max()
test_dates_df = date_info_df[date_info_df['show_date'] > max_train_date]
print(f"Test period identified: {test_dates_df['show_date'].min().date()} to {test_dates_df['show_date'].max().date()}")

# --- 3c: Create Test Scaffolding ---
all_theater_ids = booknow_visits_df['book_theater_id'].unique()
test_scaffold_df = pd.DataFrame(product(all_theater_ids, test_dates_df['show_date']),
                                columns=['book_theater_id', 'show_date'])
print(f"Test scaffold created with shape: {test_scaffold_df.shape}")

# --- 3d: Combine Train and Test ---
# `audience_count` will be NaN for the test pairs
full_data_df = pd.concat([booknow_visits_df, test_scaffold_df], sort=True)
full_data_df = full_data_df.sort_values(by=['book_theater_id', 'show_date']).reset_index(drop=True)

# --- 3e: Re-run Feature Engineering on Full Dataset ---
# We re-use the aggregated DataFrames from Step 1
print("Merging all features into full dataset...")
full_master_df = pd.merge(full_data_df, date_info_df, on='show_date', how='left')
full_master_df = pd.merge(full_master_df, booknow_agg_df, on=['book_theater_id', 'show_date'], how='left')
full_master_df = pd.merge(full_master_df, cinepos_linked_agg_df, on=['book_theater_id', 'show_date'], how='left')

# Cleanup NaNs
full_master_df['total_booknow_tickets'].fillna(0, inplace=True)
full_master_df['total_cinepos_tickets'].fillna(0, inplace=True)
full_master_df['total_tickets'] = full_master_df['total_booknow_tickets'] + full_master_df['total_cinepos_tickets']

# Date Features
full_master_df['day_of_month'] = full_master_df['show_date'].dt.day
full_master_df['month'] = full_master_df['show_date'].dt.month
full_master_df['year'] = full_master_df['show_date'].dt.year
full_master_df['day_of_year'] = full_master_df['show_date'].dt.dayofyear
full_master_df['is_weekend'] = full_master_df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)

# Lag & Rolling Features
# This now correctly uses train data to create lags for the test data
print("Creating lags on full dataset...")
gb_full = full_master_df.groupby('book_theater_id')['audience_count']
full_master_df['audience_lag_7'] = gb_full.shift(7)
full_master_df['audience_lag_14'] = gb_full.shift(14)
full_master_df['audience_roll_mean_7'] = gb_full.shift(1).rolling(7, min_periods=1).mean()

# Categorical Encoding
full_master_df['book_theater_id_encoded'] = le.transform(full_master_df['book_theater_id']) # Use the LE from Step 2
full_master_df = pd.get_dummies(full_master_df, columns=['day_of_week'], prefix='dow')

print("--- Full Train+Test Dataset is Ready ---")
print(full_master_df.info())

Step 4: Final Model Training & Submission

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

print("\n--- Starting Step 4: Final Training & Submission ---")

# --- 4a: Split into Final Train and Test ---
# Training data is where 'audience_count' is known
train_final_df = full_master_df[full_master_df['audience_count'].notnull()]
# Test data is where 'audience_count' is unknown
test_final_df = full_master_df[full_master_df['audience_count'].isnull()]

# Clean the final training data (drop initial NaNs)
train_final_df = train_final_df.dropna(subset=['audience_lag_7', 'audience_lag_14', 'audience_roll_mean_7'])

print(f"Final training data shape: {train_final_df.shape}")
print(f"Final test data shape: {test_final_df.shape}")

# --- 4b: Align Columns ---
# Get feature list from the training set
features = [col for col in train_final_df.columns if col not in [
    'audience_count', 'show_date', 'book_theater_id'
]]

# Ensure test set has the exact same columns as the train set
X_train_final = train_final_df[features]
y_train_final = train_final_df[target_col]

# Align test set columns
X_test_final = test_final_df.copy()
for col in features:
    if col not in X_test_final.columns:
        X_test_final[col] = 0
X_test_final = X_test_final[features] # Keep only feature columns in correct order

# Handle any NaNs in test features (e.g., if a new theater had no lag data)
# For this problem, we'll fill with 0
X_test_final.fillna(0, inplace=True)

# --- 4c: Train Final Model ---
print("Training final model on ALL available data...")
rf_final = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=5,
    max_features=0.7
)
rf_final.fit(X_train_final, y_train_final)
print("Final model trained.")

# --- 4d: Make Predictions ---
print("Making final predictions...")
predictions = rf_final.predict(X_test_final)

# --- 4e: Format Submission File ---
submission_df = test_final_df[['book_theater_id', 'show_date']].copy()
submission_df['audience_count'] = predictions

# Format the ID: book_theater_id + show_date
submission_df['show_date'] = submission_df['show_date'].dt.strftime('%Y-%m-%d')
submission_df['ID'] = submission_df['book_theater_id'] + '_' + submission_df['show_date']

# Ensure predictions are non-negative and integers
submission_df['audience_count'] = np.round(submission_df['audience_count']).astype(int)
submission_df.loc[submission_df['audience_count'] < 0, 'audience_count'] = 0

# Select final columns
final_submission = submission_df[['ID', 'audience_count']]

# Save the file
final_submission.to_csv('submission.csv', index=False)

print("\n--- Submission File Created! ---")
print(final_submission.head())
print(f"File 'submission.csv' saved with {len(final_submission)} predictions.")